# Import Libraries

In [21]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from login_info import cr_username
from login_info import cr_password
import time

# Obtain Data

Create Selenium webdriver

In [106]:
driver_path = 'chromedriver/chromedriver'
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options, executable_path=driver_path)

Log into Consumer Reports website

In [107]:
driver.get('https://www.consumerreports.org/cro/modal-login/index.htm')
driver.find_element_by_xpath('/html/body/div/div/div/div/div[1]/form/div[1]/input').send_keys(cr_username)
driver.find_element_by_xpath('/html/body/div/div/div/div/div[1]/form/div[2]/input').send_keys(cr_password)
driver.find_element_by_xpath('/html/body/div/div/div/div/div[1]/form/div[6]/input').click()

Scrape Consume Reports vehicle classees

In [24]:
driver.get('https://www.consumerreports.org/cars/types/used/convertibles')
classes_html = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div').get_attribute('innerHTML')
vehicle_class_soup = BeautifulSoup(classes_html, 'html.parser')
vehicle_classes=['Convertibles']
for element in vehicle_class_soup.find_all('li'):
    vehicle_classes.append(element.text.replace('\n', ''))
vehicle_classes

['Convertibles',
 'Hybrids/EVs',
 'Luxury Cars',
 'Minivans',
 'Pickup Trucks',
 'Sedans',
 'Small Cars',
 'Sports Cars',
 'SUVs',
 'Wagons']

Create list of vehicle class urls

In [25]:
vehicle_class_urls=[]
for vehicle_class in vehicle_classes:
    formatted_vehicle_class = vehicle_class.replace('/', '-').replace(' ', '-').lower()
    vehicle_class_urls.append(f'https://www.consumerreports.org/cars/types/used/{formatted_vehicle_class}')
vehicle_class_urls

['https://www.consumerreports.org/cars/types/used/convertibles',
 'https://www.consumerreports.org/cars/types/used/hybrids-evs',
 'https://www.consumerreports.org/cars/types/used/luxury-cars',
 'https://www.consumerreports.org/cars/types/used/minivans',
 'https://www.consumerreports.org/cars/types/used/pickup-trucks',
 'https://www.consumerreports.org/cars/types/used/sedans',
 'https://www.consumerreports.org/cars/types/used/small-cars',
 'https://www.consumerreports.org/cars/types/used/sports-cars',
 'https://www.consumerreports.org/cars/types/used/suvs',
 'https://www.consumerreports.org/cars/types/used/wagons']

Get html for each vehicle class url with list of vehicles for that class

In [26]:
soups = []
for url in vehicle_class_urls:
    driver.get(url)
    time.sleep(1)
    table_html = driver.find_element_by_xpath('/html/body/div[4]/div/div[2]/div/div[2]/table/tbody').get_attribute('innerHTML')
    soups.append(BeautifulSoup(table_html,'html.parser'))

Create list of all Consumer Reports rated used vehicles from vehicle class urls html

In [27]:
cars_list = []
for soup in soups:
    for item in soup.find_all('tr'):
        if len(item.find_all('td', {'rowspan': '4'}))>0:
            model = item.find('td').find('div').find('div').find('div').find('a').find('span').text
            brand = item.find('td').find('div').find('div').find('div').find('a').text.replace(model, '')
            cars_list.append(brand+':'+model)

cars_list = list(set(cars_list))
cars_list[0:10]

['Cadillac:CT4',
 'Infiniti:FX',
 'Acura:TL',
 'Volvo:V60',
 'Fiat:500L',
 'Toyota:Yaris iA',
 'Dodge:Dakota',
 'Jaguar:XF',
 'Audi:TT',
 'Toyota:Mirai']

Create list of urls to scrape from cars list

In [28]:
reviews_urls_list = []
for car in cars_list:
    car = car.replace(' ', '-').lower()
    car_brand = car.split(':')[0]
    car_model = car.split(':')[1]
    reviews_urls_list.append(f'https://www.consumerreports.org/cars/{car_brand}/{car_model}/2019/overview/')
reviews_urls_list[0:10]

['https://www.consumerreports.org/cars/cadillac/ct4/2019/overview/',
 'https://www.consumerreports.org/cars/infiniti/fx/2019/overview/',
 'https://www.consumerreports.org/cars/acura/tl/2019/overview/',
 'https://www.consumerreports.org/cars/volvo/v60/2019/overview/',
 'https://www.consumerreports.org/cars/fiat/500l/2019/overview/',
 'https://www.consumerreports.org/cars/toyota/yaris-ia/2019/overview/',
 'https://www.consumerreports.org/cars/dodge/dakota/2019/overview/',
 'https://www.consumerreports.org/cars/jaguar/xf/2019/overview/',
 'https://www.consumerreports.org/cars/audi/tt/2019/overview/',
 'https://www.consumerreports.org/cars/toyota/mirai/2019/overview/']

Assign missing_urls and review_data variables to empty lists

In [29]:
missing_urls=[]
review_data = []

Scrape data from review urls

In [108]:
review_index = 0
for review_url in reviews_urls_list[review_index:]:
    driver.get(review_url)
    if driver.find_element_by_xpath('/html').text[0:14] == 'Page Not Found':
        missing_urls.append(review_url)
        print(f'{review_index} Bad: ', review_url)
        review_index += 1
    else:
        model_data = []
        model_data.append(review_url.split('/')[-5]+'-'+review_url.split('/')[-4])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div/div/div/div/div[1]/div[1]/a/div[1]').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div/div/div/div/div[1]/div[1]/a/div[2]/div/div/div[2]').text.strip()])
        for index in [2, 7]:
            model_data.extend([driver.find_element_by_xpath(f'/html/body/div[7]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[{index}]/div[1]').text,
               driver.find_element_by_xpath(f'/html/body/div[7]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[{index}]/div[2]/div/div/div/div/div[2]/span').text])
        for index in [2, 4, 6, 8, 10, 12]:
            model_data.extend([driver.find_element_by_xpath(f'/html/body/div[7]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[{index}]/div[1]').text,
               driver.find_element_by_xpath(f'/html/body/div[7]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[{index}]/div[2]/div/div/div/div/div[2]/span').text])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[7]/div[1]/div[1]/div[2]/div[3]/div[2]/div/div[2]/div[1]').text,
            driver.find_element_by_xpath('/html/body/div[7]/div[1]/div[1]/div[2]/div[3]/div[2]/div/div[2]/div[2]/div/div/div/div/div[2]/span').text])
        time.sleep(1)
        driver.get(review_url.replace('overview/', 'ratings-specs/?pagestop'))
        time.sleep(20)
        driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/h3/div[2]').click()
        driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[7]/h3/div[2]').click()
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div/div[3]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div/div[3]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div/div[7]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div/div[7]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[3]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[3]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[6]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[6]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[10]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[10]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[7]/div/div/div[7]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[7]/div/div/div[7]/div[2]/div').text.strip()])
        review_data.append(model_data)    
        print(f'{review_index} Good: ', review_url)
        review_index += 1

507 Good:  https://www.consumerreports.org/cars/chevrolet/malibu/2019/overview/
508 Good:  https://www.consumerreports.org/cars/dodge/charger/2019/overview/


View scrape results

In [109]:
scrape_df = pd.DataFrame(review_data)
column_names = ['Model']
column_names.extend([scrape_df.loc[0].values[x//2*2-1] for x in range(2, len(scrape_df.loc[0].values)+1)])
scrape_df.columns = column_names
scrape_df.iloc[:, 0:15]

,Model,Reliability,Reliability,Acceleration,Acceleration,Braking,Braking,Ride,Ride,Noise,Noise,Front Seat Comfort,Front Seat Comfort,Rear Seat Comfort,Rear Seat Comfort
0,volvo-v60,Reliability,1 / 5,Acceleration,NA,Braking,NA,Ride,NA,Noise,NA,Front Seat Comfort,NA,Rear Seat Comfort,NA
1,fiat-500l,Reliability,NA,Acceleration,3 / 5,Braking,4 / 5,Ride,2 / 5,Noise,3 / 5,Front Seat Comfort,3 / 5,Rear Seat Comfort,4 / 5
2,jaguar-xf,Reliability,NA,Acceleration,5 / 5,Braking,5 / 5,Ride,5 / 5,Noise,4 / 5,Front Seat Comfort,5 / 5,Rear Seat Comfort,4 / 5
3,audi-tt,Reliability,NA,Acceleration,5 / 5,Braking,5 / 5,Ride,3 / 5,Noise,3 / 5,Front Seat Comfort,5 / 5,Rear Seat Comfort,1 / 5
4,toyota-mirai,Reliability,NA,Acceleration,3 / 5,Braking,3 / 5,Ride,4 / 5,Noise,4 / 5,Front Seat Comfort,4 / 5,Rear Seat Comfort,3 / 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,lexus-es,Reliability,3 / 5,Acceleration,4 / 5,Braking,4 / 5,Ride,5 / 5,Noise,5 / 5,Front Seat Comfort,5 / 5,Rear Seat Comfort,3 / 5
291,alfa-romeo-stelvio,Reliability,NA,Acceleration,4 / 5,Braking,4 / 5,Ride,4 / 5,Noise,4 / 5,Front Seat Comfort,3 / 5,Rear Seat Comfort,3 / 5
292,volkswagen-jetta,Reliability,3 / 5,Acceleration,3 / 5,Braking,4 / 5,Ride,4 / 5,Noise,4 / 5,Front Seat Comfort,4 / 5,Rear Seat Comfort,4 / 5
293,chevrolet-malibu,Reliability,NA,Acceleration,4 / 5,Braking,4 / 5,Ride,4 / 5,Noise,4 / 5,Front Seat Comfort,3 / 5,Rear Seat Comfort,4 / 5


In [110]:
scrape_df.iloc[:, 15:]

,Interior Fit and Finish,Interior Fit and Finish,Trunk/Cargo Area,Trunk/Cargo Area,Fuel Economy,Fuel Economy,Routine Handling,Routine Handling,Headlights,Headlights,Driving Position,Driving Position,Third Seat Comfort,Third Seat Comfort,Usability,Usability,Max. Load,Max. Load
0,Interior Fit and Finish,NA,Trunk/Cargo Area,NA,Fuel Economy,NA,Routine Handling,NA,Headlights,NA,Driving Position,NA,Third Seat Comfort,NA,Usability,NA,Max. Load,NA
1,Interior Fit and Finish,3 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,4 / 5,Routine Handling,4 / 5,Headlights,3 / 5,Driving Position,3 / 5,Third Seat Comfort,NA,Usability,3 / 5,Max. Load,860 lb.
2,Interior Fit and Finish,5 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,3 / 5,Routine Handling,5 / 5,Headlights,3 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,3 / 5,Max. Load,960 lb.
3,Interior Fit and Finish,5 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,4 / 5,Routine Handling,5 / 5,Headlights,2 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,2 / 5,Max. Load,770 lb.
4,Interior Fit and Finish,4 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,5 / 5,Routine Handling,2 / 5,Headlights,4 / 5,Driving Position,3 / 5,Third Seat Comfort,NA,Usability,3 / 5,Max. Load,690 lb.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,Interior Fit and Finish,5 / 5,Trunk/Cargo Area,3 / 5,Fuel Economy,4 / 5,Routine Handling,4 / 5,Headlights,2 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,2 / 5,Max. Load,905 lb.
291,Interior Fit and Finish,4 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,3 / 5,Routine Handling,5 / 5,Headlights,4 / 5,Driving Position,3 / 5,Third Seat Comfort,NA,Usability,2 / 5,Max. Load,990 lb.
292,Interior Fit and Finish,3 / 5,Trunk/Cargo Area,4 / 5,Fuel Economy,5 / 5,Routine Handling,4 / 5,Headlights,2 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,5 / 5,Max. Load,970 lb.
293,Interior Fit and Finish,3 / 5,Trunk/Cargo Area,4 / 5,Fuel Economy,4 / 5,Routine Handling,4 / 5,Headlights,4 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,5 / 5,Max. Load,900 lb.


Save scrape results to csv file

In [111]:
scrape_df.to_csv('Data/Ratings.csv')